# Structural Unit

Structural Unit 是基于 Basic Unit 构建的中层模块，负责模拟神经元的空间结构与局部功能整合。其核心包括 **SingleCompartment**、**MultiCompartment** 和 **HHTypeNeuron**，三者共同构成从局部膜区域到完整神经元的结构建模体系。


## 一、SingleCompartment：神经元局部区域的简化模拟

### 1.1 生物学背景
真实神经元的形态复杂，但在许多场景下，可将神经元的局部区域视为单房室——一个**均匀的、无空间差异的膜区域**，忽略其内部的电流空间衰减。

这种简化源于神经科学中的电缆理论：当研究区域的长度远小于电信号的空间常数时，该区域的膜电位可近似为均匀分布，无需考虑空间差异。


### 1.2 物理学机制
单隔室的核心是**膜电位的时间动态**，其物理模型可简化为RC电路：
- **膜电容（Cₘ）**：由隔室的表面积和膜电容密度决定（Cₘ = 面积 × 比电容），描述膜储存电荷的能力（充电速度）
- **膜电阻（Rₘ）**：由膜上离子通道的总电导决定（Rₘ = 1/总电导），描述离子流动的阻力
- **电流平衡**：膜电位变化由跨膜电流决定，遵循方程：
  $$ C_m \frac{dV}{dt} = I_{\text{input}} - I_{\text{ionic}} $$
  其中，$ I_{\text{input}} $ 是外部输入电流，$ I_{\text{ionic}} $ 是离子通道产生的总电流（由 Basic Unit 中的离子通道计算）。


## 二、MultiCompartment：神经元空间结构的精确建模

### 2.1 生物学背景
当神经元区域的长度接近或超过空间常数时，电信号会随传播距离衰减，单隔室模型不再适用。多隔室模型通过**将神经元分割为多个串联或并联的单隔室**，模拟真实神经元的空间形态，准确捕捉信号的空间传播特性。



### 2.2 物理学机制
多房室模型基于**电缆方程**，核心是房室间的**轴向电流**与**膜电流的耦合**：
- **轴向电阻（Rₐ）**：描述电流在房室之间流动的阻力（与隔室长度成正比，与横截面积成反比）
- **房室耦合**：相邻房室的膜电位差驱动轴向电流，遵循
  $$ I_{\text{axial}} = \frac{V_j - V_i}{R_a} $$
  （\( V_i \)、\( V_j \) 为相邻房室的膜电位）
- **整体动态**：每个房室的膜电位变化同时受自身离子电流和相邻房室的轴向电流影响，形成微分方程组，准确模拟信号的空间传播与衰减


## 三、HHTypeNeuron：经典动作电位模型的封装

### 3.1 生物学背景
Hodgkin-Huxley 模型是描述动作电位产生机制的经典模型，基于鱿鱼巨轴突的实验数据建立，揭示了“Na⁺ 内流触发去极化、K⁺ 外流导致复极化”的核心原理。HHTypeNeuron 是对这一模型的结构化封装，属于**预设离子通道组合的单房室模型**，专为模拟动作电位的产生与传导设计。


### 3.2 物理学机制
HHTypeNeuron 的核心是**离子电流的协同作用**，基于 Basic Unit 中的 Na⁺ 通道、K⁺ 通道和漏通道：
- **Na⁺ 电流（Iₙₐ）**：由快速激活门（m）和缓慢失活门（h）控制，负责动作电位的上升相
- **K⁺ 电流（Iₖ）**：由延迟激活门（n）控制，负责动作电位的下降相和超极化
- **漏电流（Iₗₑₐₖ）**：恒定的非选择性电流，维持静息电位
- **膜电位动态**：总电流驱动膜电位变化，方程为：
  $$ C_m \frac{dV}{dt} = I_{\text{input}} - (I_{Na} + I_K + I_{leak}) $$


## 四、关系：从局部到整体的结构建模逻辑

### 4.1 层级依赖
- **HHTypeNeuron 是特殊的 SingleCompartment**：HHTypeNeuron 本质是预设了“Na⁺ 通道 + K⁺ 通道 + 漏通道”组合的单房室，专注于动作电位模拟
- **MultiCompartment 由 SingleCompartment 组成**：多房室模型通过轴向电阻连接多个单房室（可包含 HHTypeNeuron 或自定义单房室），形成复杂空间结构


### 4.2 功能互补
- **SingleCompartment**：用于简化场景，快速验证离子通道组合的电响应
- **MultiCompartment**：用于研究空间因素（如树突整合、轴突传导）对神经元功能的影响
- **HHTypeNeuron**：用于精准模拟动作电位，复现经典神经电生理现象



### 4.3 建模流程
从 Basic Unit 到 Structural Unit 的构建逻辑：
1. 用 Basic Unit 定义离子和离子通道
2. 将通道添加到 SingleCompartment 中，定义局部膜特性
3. 若需模拟动作电位，直接使用预配置的 HHTypeNeuron
4. 若需模拟空间结构，将多个 SingleCompartment通过轴向电阻连接，构建 MultiCompartment

通过这种层级结构，Structural Unit 实现了从局部膜区域到完整神经元的灵活建模，兼顾简化性与生物真实性。